## Herzlich willkommen zum Packaging Valley Makeathon bei Bosch Rexroth

Willkommen zum Packaging Valley Makeathon! Hier habt ihr die Chance, innovative Lösungen für die Fabrik der Zukunft zu entwickeln und eure Kreativität mit modernster Technologie zu verbinden. Gemeinsam gestalten wir die nächste Generation der industriellen Automation und bringen eure Ideen direkt in die Praxis.

Das Ziel des Makeathons ist es, ein System zu entwickeln, das es Anwendern ermöglicht, Mover im FLOW6D-System über natürlichsprachliche Anfragen zu steuern. Dabei soll ein Large Language Model (LLM) wie Gemini 2.5 Pro genutzt werden, um die Anfragen zu interpretieren und in ausführbaren Python-Code umzuwandeln. Dieser Code wird zuerst in der Simulation auf einer virtuellen ctrlX Core getestet und danach auf dem echten System zum Laufen gebracht.

Lasst euch inspirieren, experimentiert und arbeitet als Team zusammen – eure Lösungen können echten Mehrwert schaffen und die Zukunft der Produktion mitgestalten!


### Einführung in die Tools des Makeathons

#### Jupyter Notebook
**Ansprechpartner:** René Kübler

Das Jupyter Notebook ist eine interaktive Umgebung, in der ihr Python-Code direkt ausführen und mit erklärenden Texten kombinieren könnt. Während des Makeathons nutzt ihr das Notebook, um über eine API mit einem Large Language Model (LLM) wie Gemini 2.5 Pro zu kommunizieren.

**Kurzanleitung: Ausführen und Bearbeiten von Codezellen**
Ihr könnt die Zellen im Notebook jederzeit bearbeiten, um euren Code oder die Texte anzupassen. Damit eure Änderungen wirksam werden, müsst ihr die jeweilige Zelle nach der Bearbeitung erneut ausführen. Erst durch das Ausführen einer Zelle werden die neuen Inhalte übernommen und der Code ausgeführt. Klickt dazu in die gewünschte Zelle und drückt <Shift> + <Enter> oder nutzt den "Run"-Button (▶) oben im Menü. Das Ergebnis erscheint direkt unter der Zelle.

Im weiteren Verlauf des Notebooks werden die Funktionsweise der Codeblöcke und die Aufgabenstellungen erläutert. Ziel ist es, euch ein besseres Verständnis für die Lösungen zu vermitteln und euch hilfreiche Hinweise zu geben. Bei Fragen stehen euch die genannten Ansprechpartner sowie das gesamte Team gerne zur Verfügung.





#### ctrlX Works

**Ansprechpartner:** Julian Schill, René Kübler


ctrlX Works ist die Engineering-Plattform für ctrlX Automation. Die ctrlX Core ist eine Speicherprogrammierbare Steuerung (SPS), die in der Modellfabrik verschiedene Maschinen steuert, darunter das ctrlX FLOW6D-System.


Im Makeathon wird ctrlX Works verwendet, um eine virtuelle ctrlX Core bereitzustellen. Damit kann Python-Code auf einer simulierten Steuerung getestet werden, bevor er auf die reale Hardware übertragen wird. Dies ermöglicht eine sichere und effiziente Entwicklung und Fehleranalyse.


Die Plattform bietet eine Entwicklungsumgebung mit Tools zur Integration von Apps und Services aus dem ctrlX OS Store. Eigene Erweiterungen können programmiert und getestet werden. Die Community unterstützt bei technischen Fragen und der Weiterentwicklung von Lösungen.


#### ctrlX FLOW6D

**Ansprechpartner:** Julian Schill


ctrlX FLOW6D ist ein Planarsystem mit sechs Freiheitsgraden. Die freischwebenden Transport-Plattformen, sogenannte Mover, bewegen sich auf einer modular anpassbaren Arbeitsfläche – horizontal, vertikal oder überkopf. Jeder Mover kann in sechs Dimensionen gesteuert werden: Schweben, Kippen, Drehen und Verschieben mit hoher Geschwindigkeit und Präzision, ganz ohne Reibung oder Verschmutzung.


Im Makeathon steuert ihr die Mover über die ctrlX Core und könnt so komplexe Bewegungsabläufe und Interaktionen realisieren. Die Integration in ctrlX AUTOMATION ermöglicht eine kompakte Steuerung und die Erweiterung des Funktionsumfangs durch Apps. 

### Implementierung der API-Kommunikation
#### 1. Installation und Einbindung der benötigten Bibliotheken
Im ersten Codeblock werden die wichtigsten Bibliotheken für die Arbeit im Notebook installiert und eingebunden: `requests` und `json`.

* **`requests`**: Mit dieser Bibliothek könnt ihr HTTP-Anfragen stellen. Wir benutzen sie, um mit einer API zu kommunizieren.
* **`json`**: Diese Bibliothek hilft euch beim Umgang mit JSON-Daten (JavaScript Object Notation). JSON ist ein weit verbreitetes, leichtgewichtiges Format zum Speichern und Übertragen von strukturierten Daten. Es eignet sich sowohl für Konfigurationsparameter als auch für komplexe Datenstrukturen und wird häufig als Ausgabeformat bei Tests verwendet.

In [ ]:
!pip install requests

In [ ]:
import requests # For the http request
import json # For the transfer of the parameters
import time # Calculating the time from the web request

#### 2. API-Konfiguration und Authentifizierung

In diesem Abschnitt werden die wichtigsten Parameter für die Kommunikation mit dem Google Cloud Gateway festgelegt. Die Zugangsdaten und Schlüssel werden dabei automatisch aus der Datei `PV_Key.json` geladen. Über dieses Gateway erfolgt der Zugriff auf das Large Language Model.

* **`API_GATEWAY_URL`**: Die Basis-URL des Gateways, an die alle API-Anfragen zum Gemini Pro Modell gesendet werden.
* **`GATEWAY_API_KEY`**: Der Schlüssel zur Authentifizierung gegenüber dem Google Cloud Gateway. Er bestätigt, dass eure Anwendung berechtigt ist, das Gateway zu nutzen.
* **`USER_API_KEY`**: Euer persönlicher Schlüssel, der euch als Nutzer identifiziert und eure individuellen Anfragen zuordnet.

Alle Schlüssel und Einstellungen werden gemeinsam im `headers`-Dictionary gebündelt. Dieses Dictionary wird bei jeder API-Anfrage mitgeschickt und sorgt dafür, dass die Kommunikation korrekt und sicher abläuft.

In [ ]:
# Lese API-Keys und die URL aus PV_Key.json
with open("PV_Key.json", "r") as f:
    keys = json.load(f)
API_GATEWAY_URL = keys["gateway"]["API_GATEWAY_URL"]
GATEWAY_API_KEY = keys["gateway"]["GATEWAY_API_KEY"]
USER_API_KEY = keys["users"]["USER_API_KEY_1"]  # Bei Bedarf anderen Schlüssel wählen

headers = {
    "Content-Type": "application/json",
    "x-api-key": GATEWAY_API_KEY,
    "x-user-api-key": USER_API_KEY,
    "LLM_parameters": "False" # auf "True" setzen, um die Parameter zu benutzen
}


#### 3. Nutzerprompt

Im folgenden Abschnitt könnt ihr eure Anfrage an Gemini 2.5 Pro eingeben. Dieser Prompt wird als Eingabe für das Large Language Model dienen, um die gewünschte Antwort zu generieren.

Hinweis: In Python werden drei einfache Anführungszeichen (`'''`) verwendet, um mehrzeilige Strings zu definieren. Das ist besonders praktisch, wenn ihr längere Prompts oder Texte als Eingabe verwenden möchtet.

In [ ]:
prompt_text = '''Write a \"Hello World\" program in Python.'''

## 4. Konfiguration der API-Anfrage (Payload)

Dieser Codeblock definiert den `payload` (die "Nutzlast"), der die tatsächlichen Daten und Einstellungen für die Anfrage an das Gemini 2.5 Pro Modell enthält.

* **`prompt`**: Der eigentliche Text, den ihr als Anweisung oder Frage an das Large Language Model sendet. Hier wird der zuvor definierte `prompt_text` eingesetzt.
* **`temperature`**: Steuert die Kreativität und Zufälligkeit der generierten Antwort.
    * Ein Wert nahe 0 (z. B. 0.0) führt zu sehr deterministischen und konservativen Antworten (weniger Variation).
    * Ein höherer Wert (z. B. 1.0) führt zu kreativeren und diverseren Antworten, die aber auch weniger vorhersehbar sein können.
    * **Empfehlung:** Für die meisten Anwendungsfälle liegt ein guter Startwert oft zwischen 0.5 und 0.9.
* **`top_p`**: Auch bekannt als "Nucleus Sampling". Dieser Parameter begrenzt die Auswahl der nächsten Wörter auf eine bestimmte kumulative Wahrscheinlichkeitsverteilung.
    * Ein Wert von 1.0 (Standard) bedeutet, dass alle Wörter berücksichtigt werden.
    * Ein Wert von 0.9 bedeutet, dass nur die Top-Wörter berücksichtigt werden, deren kumulierte Wahrscheinlichkeit 90 % erreicht. Dies kann helfen, weniger plausible Wörter zu vermeiden und die Kohärenz zu verbessern.
* **`top_k`**: Limitiert die Auswahl der nächsten Wörter auf die `k` wahrscheinlichsten Optionen.
    * Ein Wert von 1 würde nur das wahrscheinlichste Wort auswählen.
    * Ein höherer Wert (z. B. 32) erweitert die Auswahl auf die 32 wahrscheinlichsten Wörter.
    * Dies kann dazu beitragen, die Vielfalt der generierten Antworten zu erhöhen, während gleichzeitig unpassende Wörter ausgeschlossen werden.
* **`candidate_count`**: Gibt an, wie viele verschiedene Antwortvorschläge (Kandidaten) das Modell generieren soll.
    * Ein Wert von 1 (Standard) bedeutet, dass nur eine Antwort generiert wird.
    * Höhere Werte können nützlich sein, um verschiedene Formulierungen oder Ansätze des Modells zu vergleichen.
* **`max_output_tokens`**: Definiert die maximale Länge der vom Modell generierten Antwort, gemessen in Tokens. Ein Token kann ein Wort, ein Teil eines Wortes oder ein Satzzeichen sein.
    * Ein höherer Wert erlaubt längere Antworten, kann aber auch die Rechenzeit erhöhen.
    * Stellt sicher, dass das Modell genügend "Platz" hat, um eine vollständige Antwort zu generieren.

In [ ]:
payload = {
    "prompt": prompt_text,
    "temperature"       : 0.9,
    "top_p"             : 1.0,
    "top_k"             : 32,
    "candidate_count"   : 1,
    "max_output_tokens" : 100000
}


#### 5. Ausführen der API-Anfrage und Fehlerbehandlung

In diesem Abschnitt wird die konfigurierte Anfrage an die Gemini Pro API gesendet und mögliche Fehler werden zuverlässig behandelt.

Der gesamte Ablauf ist in einem `try...except`-Block gekapselt, um auf Probleme während der Kommunikation gezielt reagieren zu können.

* **`try` Block:**
    * **`response = requests.post(API_GATEWAY_URL, headers=headers, data=json.dumps(payload))`**: Sendet die POST-Anfrage an das API-Gateway.
        * `API_GATEWAY_URL`: Die zuvor festgelegte URL des Gateways.
        * `headers`: Das Dictionary mit den Authentifizierungsdaten.
        * `data=json.dumps(payload)`: Der konfigurierte Payload (Prompt und Parameter) wird als JSON-Datenkörper übermittelt.
    * **`response.raise_for_status()`**: Prüft den Statuscode der Serverantwort. Bei einem Fehler (4xx oder 5xx) wird automatisch eine Ausnahme ausgelöst.
    * **`print("STATUSCODE:", response.status_code)`**: Gibt den HTTP-Statuscode der Antwort aus (erwartet wird `200` für "OK").
    * **`print(response.text)`**: Zeigt den eigentlichen Textinhalt der Serverantwort, also die generierte Antwort von Gemini 2.5 Pro.

* **`except requests.exceptions.HTTPError as e` Block:**
    * Wird ausgeführt, wenn ein HTTP-Fehler erkannt wurde (z. B. falsche Authentifizierung oder Serverfehler).
    * Gibt detaillierte Informationen zum Fehler aus, einschließlich Statuscode und Servernachricht, um die Fehlersuche zu erleichtern.

* **`except requests.exceptions.RequestException as e` Block:**
    * Fängt alle anderen Arten von `requests`-spezifischen Fehlern ab, z. B. Netzwerkprobleme, DNS-Fehler oder Timeouts.
    * Informiert den Nutzer über einen allgemeinen Verbindungsfehler.

**Hinweise:**

- Diese strukturierte Fehlerbehandlung sorgt für eine stabile Anwendung und liefert bei Problemen hilfreiche Rückmeldungen.
- Die API-Anfragen können je nach Komplexität des Prompts und der Serverauslastung recht lange dauern. Daher wurde ein großzügiges Timeout von 300 Sekunden (5 Minuten) eingestellt, um sicherzustellen, dass auch umfangreiche Anfragen bearbeitet werden können.


In [ ]:
try:
    start_time = time.time()
    # Make a POST request
    response = requests.post(
        API_GATEWAY_URL,
        headers=headers,
        data=json.dumps(payload),
        timeout=300  # Timeout in seconds (5 minutes),
    )

    # Check for 4xx or 5xx errors
    response.raise_for_status()

    print("\n--- SUCCESS! ---")
    print("STATUS CODE:", response.status_code)  # Should be 200
    print("\nResponse from Gemini Model:")
    print(response.text)
    print("------------------------------------------")
    print("\nThe system is working successfully.")
    end_time = time.time()
    time_diff = end_time - start_time
    print(f"Time for response: {time_diff}")

except requests.exceptions.HTTPError as e:
    # Handles errors like 404, 403, 500
    print("\n--- An error occurred ---")
    print(f"Status code: {e.response.status_code}")
    print(f"Server message: {e.response.text}")
    print("---------------------------------")

except requests.exceptions.RequestException as e:
    # Handles network issues
    print(f"\nA connection error occurred: {e}")


#### Ausführen des generierten Codes

Um den generierten Python-Code auf eurem Computer auszuführen, kopiert den Code aus dem Notebook in eine neue Datei mit der Endung `.py` (z. B. `mein_code.py`). Stellt sicher, dass Python auf eurem System installiert ist. Ihr könnt den Code dann über die Kommandozeile oder PowerShell mit folgendem Befehl starten:

```powershell
python mein_code.py
```

Alternativ könnt ihr den Code direkt im Jupyter Notebook in einer neuen Codezelle ausführen. Achtet darauf, dass alle benötigten Bibliotheken installiert sind (z. B. mit `pip install requests`). So könnt ihr den generierten Code einfach testen und bei Bedarf anpassen.

**Hinweis:** Vielleicht habt ihr sogar Lust, diesen Schritt zu automatisieren – zum Beispiel indem der generierte Code direkt aus dem Notebook heraus gespeichert und ausgeführt wird. Das ist optional, aber eine spannende Möglichkeit, eure Lösung noch komfortabler und innovativer zu gestalten!

**Noch ein Hinweis:** Ihr dürft das Programm für die API-Anfragen auch als reines Python-Programm umsetzen und müsst nicht zwingend Jupyter Notebook verwenden. Ihr könntet sogar eine einfache Benutzeroberfläche (GUI) erstellen, um die Eingabe des Prompts und die Anzeige der Antwort zu erleichtern. Wählt einfach die Umgebung, die für euch am besten passt. Vielleicht wollt ihr ja sogar Gemini 2.5 Pro benutzen um das umzusetzen?



#### Aufgabenbeschreibung

Die folgenden Aufgaben sind als Einstieg und Inspiration gedacht. Sie sollen euch helfen, die ersten Schritte zu machen und ein Gefühl für die Möglichkeiten des Systems zu bekommen. Nutzt sie als Orientierung – aber lasst eurer Kreativität freien Lauf! Ihr könnt eigene Ideen entwickeln, neue Ansätze ausprobieren und gerne experimentieren. Der Makeathon lebt von euren individuellen Lösungen.

1.1 Formuliere einen Prompt, der ein Programm erstellt, das mit der REST-API der (virtuellen) ctrlX Core interagiert. Beachte, dass der Port der virtuellen Core meist 8443 ist.

**Nützliche Links:**
- [Howto: Using REST API of ctrlX CORE](https://community.boschrexroth.com/ctrlx-automation-how-tos-qmglrz33/post/using-rest-api-of-ctrlx-core-HCE2Q8WW3uhUYOh)
- [API Beschreibung: ctrlX OS Data Layer API v2.1.3](https://boschrexroth.github.io/rest-api-description/app/?urls.primaryName=ctrlX%20OS%20-%20Data%20Layer%20API%20v2.1.3)
- [Repository der API Beschreibung](https://github.com/boschrexroth/rest-api-description/tree/master)

1.2 Formuliere einen Prompt, der ein Programm erstellt, das die verfügbaren Mover im FLOW6D-System auflistet. Schaue dazu die verfügbaren Nodes im Datalayer der virtuellen ctrlX Core an.

1.3 Formuliere einen Prompt, der ein Programm erstellt, das einen Mover von Position 1 zu Position 2 fahren lässt. Die folgenden Parameter sollen dabei angesteuert werden:
- Position x
- Position y
- Geschwindigkeit

1.4 Erweitere den Prompt so, dass zusätzlich folgende Parameter gesteuert werden:
- Wippen
- Drehen

2.1 Baue die Anfrage so um, dass du einen Nutzerprompt und einen Systemprompt verwendest. Der Nutzerprompt ändert sich bei jeder Anfrage und spiegelt die Intention des Nutzers wider. Der Systemprompt bleibt immer gleich und reguliert die Nutzerausgabe.

2.2 Schreibe einen einfachen Satz in die Nutzerausgabe, z. B.: "Ich möchte, dass der Mover von hier (x, y) zu (x, y) fährt und sich an Position (x, y) dreht." Formuliere den Systemprompt so, dass er auch unvollständige oder unkonkrete Anfragen verarbeiten kann.

2.3 Passe den Systemprompt so an, dass das System in der Lage ist, mit unterschiedlichen Hindernissen umzugehen und diese zu umfahren.

2.4 Szenario: Ein Mover wartet und steht dabei einem anderen im Weg. Erweitere den Systemprompt so, dass Mover 1 zu Mover 2 fährt, wartet, bis dieser seine Position verlassen hat, und anschließend Mover 1 an die Zielposition fährt.

3.1 Es gibt einen Hindernis-Parcours. Wie einfach kann der User-Prompt sein, damit der Mover den Parcours durchläuft?

3.2 Lasse zwei Mover miteinander interagieren, sodass beide sich parallel bewegen.

#### Prompting-Techniken und Probleme bei der Codegenerierung

Bei der Nutzung von Large Language Models (LLMs) zur Codegenerierung ist die Qualität des Prompts entscheidend für das Ergebnis. Ein gut formulierter Prompt kann zu präzisem, lauffähigem und optimiertem Code führen, während ein unklarer oder unvollständiger Prompt zu irrelevanten, fehlerhaften oder sogar unsicheren Ergebnissen führen kann.

##### Häufige Prompting-Techniken für Codegenerierung:

1.  **Zero-Shot Prompting:**
    *   **Beschreibung:** Eine direkte Anweisung, ohne Beispiele. Das LLM muss die Aufgabe aus dem Kontext und seinem Training selbstständig lösen.
    *   **Beispiel:** "Schreibe eine Python-Funktion, die zwei Zahlen addiert."
    *   **Anwendung:** Gut für einfache, klar definierte Aufgaben.

2.  **Few-Shot Prompting:**
    *   **Beschreibung:** Der Prompt enthält ein oder mehrere Beispiele für die gewünschte Eingabe und Ausgabe, um das LLM auf das gewünschte Format oder den Stil einzustimmen.
    *   **Beispiel:**
        
      Beispiel 1:
      Input: Eine Funktion, die "Hallo Welt" ausgibt. <br>
      Output: <br>
      def greet_world(): <br>
      <div style="margin-left: 2em;">print("Hallo Welt")
      </div>
      <br> Beispiel 2:
      Input: Eine Funktion, die zwei Zahlen addiert. <br>
      Output: <br>
      def add_numbers(a, b): <br>
      <div style="margin-left: 2em;"> return a + b </div> <br>

      Input: Eine Funktion, die den größten Wert in einer Liste findet.
      Output:
     
        
    *   **Anwendung:** Ideal, um das LLM an spezifische Coding-Stile, komplexe Logiken oder die Einhaltung bestimmter API-Signaturen zu gewöhnen.

3.  **Chain-of-Thought (CoT) Prompting:**
    *   **Beschreibung:** Das LLM wird angewiesen, seine Denkprozesse oder Zwischenschritte zu verbalisieren, bevor es die endgültige Antwort (den Code) generiert. Dies hilft dem Modell, komplexe Probleme schrittweise zu lösen.
    *   **Beispiel:** "Schreibe eine Python-Funktion, die prüft, ob eine Zahl prim ist. Denke Schritt für Schritt darüber nach, wie eine Primzahl definiert ist und welche Prüfungen notwendig sind, bevor du den Code schreibst."
    *   **Anwendung:** Besonders nützlich für algorithmenlastige oder mehrstufige Codierungsaufgaben.

4.  **Role-Playing / Persona Prompting:**
    *   **Beschreibung:** Das LLM nimmt die Rolle eines bestimmten Experten an (z. B. ein erfahrener Python-Entwickler, ein Sicherheitsexperte).
    *   **Beispiel:** "Du bist ein erfahrener JavaScript-Entwickler. Schreibe eine asynchrone Funktion, die Daten von einer API abruft und Fehler behandelt."
    *   **Anwendung:** Beeinflusst den Stil, die Best Practices und die Gründlichkeit des generierten Codes.

##### Häufige Probleme bei der Codegenerierung durch LLMs:

1.  **"Lost in the Middle" (Verlust im Kontext):**
    *   **Problem:** LLMs tendieren dazu, Informationen am Anfang und Ende eines längeren Prompts besser zu verarbeiten als jene in der Mitte. Bei umfangreichen Code-Anfragen oder detaillierten Spezifikationen in der Mitte des Prompts kann das LLM wichtige Details übersehen.
    *   **Lösung:** Schlüsselinformationen oder entscheidende Randbedingungen am Anfang oder Ende des Prompts platzieren. Prompts in kleinere, fokussierte Schritte unterteilen.

2.  **Unzureichende oder vage Spezifikationen:**
    *   **Problem:** Wenn der Prompt zu allgemein gehalten ist (z. B. "Schreibe eine Web-App"), fehlt dem LLM die notwendige Klarheit über die gewünschte Funktionalität, Technologie (Frameworks, Sprachen), Architektur oder Fehlerbehandlung.
    *   **Lösung:** So präzise wie möglich sein: Gebt die gewünschte Programmiersprache, Frameworks, erwartete Ein- und Ausgaben, Fehlerbehandlungsstrategien und Testfälle an.

3.  **Halluzinationen / faktische Fehler im Code:**
    *   **Problem:** Das LLM kann Code generieren, der syntaktisch korrekt aussieht, aber logische Fehler enthält, falsche APIs verwendet oder Best Practices ignoriert. Dies ist besonders kritisch bei sicherheitsrelevantem Code.
    *   **Lösung:** Prompting mit expliziten Anforderungen an Robustheit, Effizienz oder Sicherheitsaspekte. Immer den generierten Code *gründlich überprüfen und testen*, da LLMs keine Garantien für Korrektheit bieten.

4.  **Fehlende Kontextkenntnisse (Domänenwissen):**
    *   **Problem:** LLMs haben kein intrinsisches Verständnis für eure spezifische Codebasis, interne APIs, Unternehmensstandards oder komplexe Domänenlogiken. Der generierte Code passt möglicherweise nicht nahtlos in eure bestehende Infrastruktur.
    *   **Lösung:** Relevante Code-Snippets, API-Dokumentationen oder interne Richtlinien im Prompt bereitstellen (Few-Shot). Fine-Tuning des Modells mit eigenen Daten (fortgeschritten).

5.  **Formatierungs- und Layout-Probleme:**
    *   **Problem:** Obwohl LLMs Code generieren können, halten sie sich nicht immer an spezifische Code-Formatierungsstandards (PEP8 für Python, etc.), Indentierungskonventionen oder bevorzugte Kommentierungsstile.
    *   **Lösung:** Im Prompt explizit Formatierungsregeln (z. B. "Halte dich an PEP8", "Verwende 4 Leerzeichen für Einrückungen") und Kommentierungsstile vorgeben. Beispiele (Few-Shot) für den gewünschten Stil sind hier sehr effektiv.

Durch das Verständnis dieser Techniken und potenziellen Fallstricke könnt ihr die Effektivität von LLMs bei der Codegenerierung erheblich steigern und bessere, zuverlässigere Ergebnisse erzielen.

#### Informationen zum Google Cloud Backend

Für die Kommunikation mit dem Large Language Model (LLM) Gemini nutzen wir ein mehrstufiges Cloud-Setup:

- In der Google Cloud läuft ein API Gateway, das als zentrale Schnittstelle für alle Anfragen dient.
- Hinter dem Gateway befindet sich ein Function Service, der die eingehenden Anfragen entgegennimmt, verarbeitet und an das Gemini-Modell weiterleitet.
- Da wir uns in einem Unternehmensnetzwerk mit Proxy befinden, ist zusätzlich der lokale Proxy-Dienst "px" notwendig. Dieser sorgt dafür, dass die Verbindung zum Internet über den Company Proxy authentifiziert und weitergeleitet wird.

Auch wenn diese Lösung etwas umständlich erscheint, ist sie in unserem Unternehmensnetzwerk leider notwendig, um eine sichere und funktionierende Verbindung zu gewährleisten – besonders hinter dem Unternehmens-Proxy.

Informationen zum Modell: https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-pro?hl=de

Für weitere Informationen wende dich gerne an René Kübler
